**Movie Recommendation System**

Connecting Google Drive and Kaggle Datasets

In [156]:
!pip install -q kaggle

In [157]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [158]:
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat '/content/drive/MyDrive/Kaggle/kaggle.json': No such file or directory


In [159]:
! kaggle datasets download akshaydattatraykhare/movies-dataset

Dataset URL: https://www.kaggle.com/datasets/akshaydattatraykhare/movies-dataset
License(s): other
 90% 8.00M/8.89M [00:01<00:00, 13.0MB/s]
100% 8.89M/8.89M [00:01<00:00, 8.57MB/s]


In [160]:
!unzip movies-dataset.zip

Archive:  movies-dataset.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


Importing Libraries

In [161]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

Importing Data in the form of a csv file

In [162]:
df = pd.read_csv('tmdb_5000_movies.csv')

In [163]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [164]:
df = df.rename(columns={'overview': 'Description'})
df = df.rename(columns={'original_title': 'Title'})
df = df.rename(columns={'vote_average': 'Rating'})
df = df.rename(columns={'vote_count': 'Votes'})
df = df.rename(columns={'genres': 'Genre'})

In [165]:
df = df.drop(columns={'Rating','Votes','title', 'revenue','budget','homepage','id','popularity','production_companies','production_countries'})

In [166]:
df.columns

Index(['Genre', 'keywords', 'original_language', 'Title', 'Description',
       'release_date', 'runtime', 'spoken_languages', 'status', 'tagline'],
      dtype='object')

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Genre              4803 non-null   object 
 1   keywords           4803 non-null   object 
 2   original_language  4803 non-null   object 
 3   Title              4803 non-null   object 
 4   Description        4800 non-null   object 
 5   release_date       4802 non-null   object 
 6   runtime            4801 non-null   float64
 7   spoken_languages   4803 non-null   object 
 8   status             4803 non-null   object 
 9   tagline            3959 non-null   object 
dtypes: float64(1), object(9)
memory usage: 375.4+ KB


In [168]:
df.shape

(4803, 10)

In [169]:
df.columns

Index(['Genre', 'keywords', 'original_language', 'Title', 'Description',
       'release_date', 'runtime', 'spoken_languages', 'status', 'tagline'],
      dtype='object')

In [170]:
df_features = df[['Genre','Description','keywords','tagline']]

In [171]:
df_features.shape

(4803, 4)

In [172]:
df_features

,Genre,Description,keywords,tagline
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Enter the World of Pandora.
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","At the end of the world, the adventure begins."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A Plan No One Escapes
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Legend Ends
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","Lost in our world, found in another."
...,...,...,...,...
4798,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",El Mariachi just wants to play his guitar and ...,"[{""id"": 5616, ""name"": ""united states\u2013mexi...","He didn't come looking for trouble, but troubl..."
4799,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",A newlywed couple's honeymoon is upended by th...,[],A newlywed couple's honeymoon is upended by th...
4800,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",NaN
4801,[],When ambitious New York attorney Sam is sent t...,[],A New Yorker in Shanghai


In [173]:
X = df_features['Description'] + ' ' + df_features['Genre'] + ' ' + df_features['keywords'] + ' ' + df_features['tagline']

In [174]:
X

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800                                                  NaN
4801    When ambitious New York attorney Sam is sent t...
4802                                                  NaN
Length: 4803, dtype: object

In [175]:
X.shape

(4803,)

In [176]:
X = np.where(pd.isnull(X), '', X)

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(X)

In [177]:
X.shape

(4803, 30742)

Implementing Cosine Similarity Algorithm

In [178]:
from sklearn.metrics.pairwise import cosine_similarity

In [179]:
Similarity_Score = cosine_similarity(X)

In [180]:
Similarity_Score

array([[1.        , 0.34804421, 0.22578909, ..., 0.        , 0.01376301,
        0.        ],
       [0.34804421, 1.        , 0.22862935, ..., 0.        , 0.02870636,
        0.        ],
       [0.22578909, 0.22862935, 1.        , ..., 0.        , 0.01458298,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01376301, 0.02870636, 0.01458298, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [181]:
Similarity_Score.shape

(4803, 4803)

Input of Favourite Movie to check the similarity

In [182]:
Favourite_Movie_Name = input('Enter your favourite movie name : ')

Enter your favourite movie name : aquaman


In [183]:
All_Movies_Titles_List = df['Title'].tolist()

Generating Closest Title using difflib library

In [184]:
import difflib

In [185]:
Movie_Recommendation = difflib.get_close_matches(Favourite_Movie_Name,All_Movies_Titles_List)
print(Movie_Recommendation)

['Aquamarine', 'Taxman', 'Batman']


In [186]:
Close_Match = Movie_Recommendation[0]
print(Close_Match)

Aquamarine


In [187]:
Index_of_Close_Match_Movie = df[df.Title == Close_Match].index.values[0]
print(Index_of_Close_Match_Movie)

2846


In [188]:
Recommendation_Score = list(enumerate(Similarity_Score[Index_of_Close_Match_Movie]))
print(Recommendation_Score)

[(0, 0.20125187280593862), (1, 0.2052498563756079), (2, 0.12579514611575066), (3, 0.1685524198087678), (4, 0.15610940029414294), (5, 0.17522125485418139), (6, 0.15150190465449742), (7, 0.13443532595555377), (8, 0.16505655607133396), (9, 0.13514130131751279), (10, 0.0), (11, 0.12340265601070877), (12, 0.1761004604104513), (13, 0.1440066360957366), (14, 0.18444128824960393), (15, 0.170201947830986), (16, 0.14888778623017462), (17, 0.22515754369766444), (18, 0.11795352837699571), (19, 0.14279667484540434), (20, 0.14422009039554934), (21, 0.11458659383342562), (22, 0.12237147647895297), (23, 0.14389603274398965), (24, 0.13943778842216084), (25, 0.16153166792801696), (26, 0.1508193275119081), (27, 0.17544947535216238), (28, 0.15174140572446437), (29, 0.1390344341082242), (30, 0.16051770893021683), (31, 0.15635214222239865), (32, 0.14203867459805036), (33, 0.1389565066653633), (34, 0.13561506104936571), (35, 0.14695438694258087), (36, 0.11257925274000312), (37, 0.15284062387105496), (38, 0.1

In [189]:
len(Recommendation_Score)

4803

Sorting Based on Recommendation Score

In [190]:
Sorted_Similar_Movies = sorted(Recommendation_Score,key = lambda x:x[1],reverse = True)
print(Sorted_Similar_Movies)

[(2846, 1.0000000000000002), (3237, 0.2945715088714556), (2428, 0.2577127447569911), (2887, 0.2572864399020773), (2827, 0.256340077792333), (583, 0.2549030404897177), (2285, 0.2508800155570852), (3946, 0.2505997148023016), (1748, 0.2505504232439694), (4229, 0.24521205242082791), (784, 0.24282175980625947), (2442, 0.23877710064587912), (3577, 0.2373245204991752), (4436, 0.2369245366286078), (3933, 0.2342342665043593), (1874, 0.23370106266560914), (123, 0.23124185084111054), (2923, 0.2273731216025397), (2330, 0.2271956551305615), (156, 0.2268888129899537), (17, 0.22515754369766444), (2156, 0.22508638764570188), (525, 0.22491251789079184), (884, 0.2241402823472058), (2568, 0.2233624528852132), (2628, 0.22276667415733217), (3011, 0.22252460569180182), (3949, 0.2224305977088838), (106, 0.22214978886517356), (3910, 0.22201592692952832), (322, 0.22141284823220253), (3746, 0.22080747101110793), (3631, 0.22059228438615625), (196, 0.22019967330255888), (215, 0.21927604907119622), (3313, 0.218000

Recommendation List Output

In [191]:
print('Top 20 Movies Suggested for You : \n')

i = 1

for movie in Sorted_Similar_Movies:
  index = movie[0]
  title_from_index = df[df.index==index]['Title'].values[0]
  if (i<21):
    print(i, '.',title_from_index)
    i+=1

Top 20 Movies Suggested for You : 

1 . Aquamarine
2 . Splash
3 . Brooklyn's Finest
4 . Ondine
5 . Crossroads
6 . Big Fish
7 . Back to the Future
8 . Fish Tank
9 . Notes on a Scandal
10 . The To Do List
11 . In the Name of the King: A Dungeon Siege Tale
12 . Southland Tales
13 . The Way Way Back
14 . Hard Candy
15 . Jawbreaker
16 . August Rush
17 . The Matrix Revolutions
18 . St. Trinian's
19 . Mean Girls
20 . The Last Samurai


**Explanation**

In this system, we are implementing Cosine Similarity Algorithm to Recommend movies bases on the given Favourite Movie.

By using the terms: Genre, Description, Tagline and Keywords: we are able to calculate the similarites of the given favourite movie and all the movies in the dataset.
The difflib library is used to understand the user input even when it is in gibberish and make the closest assumption based on the titles available in the dataset.
Later, the similarities  are calculated based on the given terms in columns and a similarity score is generated.
This score is later sorted and the highest ones (top 20) are displayed as the Recommended Movies.